# Tutorial -  Planar Earth-Mars Transfer and Rendez-Vous
#### Thomas Goldman 


In this tutorial, I will explain how the DOLPHINN package works internally and how you can use it to solve your own problems. Realise that currently DOLPHINN is quite tailored to spacecraft trajectories. The example problem considered here, will be the optimal control problem of sending a spacecraft to meet with Mars and match its velocity by using electric propulsion. This tutorial will mainly focuss on how the DOLPHINN module works, and how it can be used to solve your own custom problem. First, lets introduce the problem. 

Consider a spacecraft with initial mass $m$ and an engine that can produce a maximum thrust force $u_{max}$ with a specific impulse $I_{sp}$. The acceleration model consists of only the central gravitional attraction from the sun. The state of the spacecraft will be described in terms of polar coordinates, $r, \theta$ and the velocity vector will be described in terms of the transverse component $v_{\theta}$ and the radial component $v_r$, as per the figure. The control vector of the thrust produced by the electric propulsion engine is described in the same basis as the velocity. However, the control is parameterized in terms of the magnitude $u_t = |\mathbf{u}|$ and an angle $u_{\phi}$. 

<img src="../Images/radial_frame.png" alt="drawing" width="400"/>


- The state of the spacecraft is then 


$$\mathbf{z} = \begin{bmatrix} r \\ \theta \\ v_r \\ v_{\theta} \end{bmatrix} $$


- The equations of motion then read

$$  \begin{align} \dot{r} &= v_r\\
    \dot{\theta} &= \dfrac{v_{\theta}}{r}\\
    \dot{v_r} &= \dfrac{v_{\theta}^2}{r}  - \dfrac{\mu}{r^2} &&+ \dfrac{u_T \sin{u_{\phi}}}{m}\\
    \dot{v_{\theta}} &= -\dfrac{v_r v_{\theta}}{r} &&+ \dfrac{u_T \cos{u_{\phi}}}{m}\label{eq:eom4}\\
    \dot{m} &= \dfrac{u_T}{I_{sp} g_0} \end{align} 
$$
    
- At the intial time $t_0$, the spacecraft is at position $\mathbf{z} (t_0) = \mathbf{z}_0$
- At the final time $t_f$, the spacecraft is at position $\mathbf{z} (t_f) = \mathbf{z}_f$

- The objective will be the propellant mass $m_p$, which can be calculated by integrating the thrust force magnitude $u_{T}$.
    $$m_p = \dfrac{1}{I_{sp} g_0} \int_{t_0}^{t_f} u_T (t) dt $$


- The solution to this optimal control problem constitutes of a trajectory $\mathbf{z}^* (t)$ and a control profile $\mathbf{u}^* (t)$, that satisfies the dynamical constraints, the boundary conditions and burns the minimum amount of fuel. 

# The DOLPHINN
The Physics-Informed Neural Network to solve this problem is summarized in this overview figure. For a detailed description of the various components in this overview, I refer to the thesis (or ASC publication). This tutorial will focuss on the implenentation in the DOLPHINN module.

<img src="../Images/final_contraints.png" alt="drawing" width="1000"/>

The DOLPHINN package uses 3 user-defined classes that together describe the basic components of the optimal control problem

1. The dynamics loss (default options in `DOLPHINN.dynamics`)
2. The constraint layer (default options in `DOLPHINN.output_layer`)
3. The objective loss (default options in `DOLPHINN.objectives`)

I wil go over these 3 and manually implement the classes for the problem described above. But first, lets quickly show how to import DOLPHINN functionality.

# Importing

- If you have DOLPHINN as a package, after downloading it via  `pip install git+https://github.com/thomas7392/DOLPHINN.git`,  you can import the DOLHPINN functionality directly 

In [ ]:
from DOLPHINN.function import Function

- If you have a (customized) local DOLPHINN clone, via `git clone git@github.com:your_username/DOLPHINN.git`, you should add the correct path to your repo to the python systems path in order to import the functionality. From the `/tutorial` folder in the repo, where we are right now, the relative path to the functionalities is `..`. 
- Depending on where you run your notebook, you should adjust this relative path

In [8]:
import sys 
import os

# Get the path to the functionality of the 
current_path = os.path.dirname(os.path.abspath('__file__'))
relative_path = '..' 
dolphinn_path = os.path.join(current_path, relative_path)

# Add the path to the DOLPHINN repo
sys.path.append(dolphinn_path)

from DOLPHINN.function import Function

# The Dynamics 

The latent neural network solution is made to adhere to the dynamical constraints, the equations of motion, by training the network on a loss that describes the equations of motion. 

- The dynamics loss class has to inherrit the `DOLPHINN.function.Function` class 
- The loss should be calculated in the user-defined (overloaded) `call` method
- The `call` method should have signature `call(self, #network_input, #network_output`)
- The derivatives in the equations of motion can be calculated with automatic differentiation by using the `deepxde.grad` module. 
- Use only `tensorflow` math operations (so no `numpy`), because this will be used by Tensorflow during training.
- You can use `self.parameter` to reach a constants named `parameter`, that might be present in the equations motion. For example $m$, $I_{sp}$ or $u_{max}$.  `DOLPHINN.function.Function` makes sure of this. You can declare their value to the DOLPHINN class later. 
\


- The class should have the following class attributes 
    - `coordinates` (str): \
    The type of coordinates. The name declared here should be in line with the coordinates transformation function in `DOLPHINN.coordinate_transformations`. Current options are `radial` and `NDcartesian`. 
    - `mass` (bool): \
    This boolean declares wheter or not the mass of the spacecraft is modelled. If set to `True`, DOLPHINN will assume that it's the last output entry from the network.
    - `control` (bool): \
    This boolean declares whether or not the spacecraft can be controlled. It set to `True`, DOLPHINN assumes that the final output entries of the network are the control entries, except when a mass is included, that they will be in front of the mass. The order is thus `[state_entries, control_entries, mass_entry]`. 
    - `entries` (int) \
    The amount of control entries plus the amount of state entries. 
     - `entry_labels` (list) \
    List with `entries` amount of strings that are used for printing purposes of the various state and control entries.   
    - `control_entries` (int) \
    The amount of control entries. 
    - `loss_entries` (int) \
    The amount of dynamics loss terms, including the potential mass loss term
    - `loss_labels` (list) \
    List with `loss_entries` amount of strings that are used for printing purposes of the various loss terms. 
    - `theta` (boolean): \
    If using polar coordinates, this declares whether or not the dynamics include theta or not. If using polar coordinates and `theta` is set to `False`, theta will be calculated afterwards from the establisehd $v_{\theta} (t)$ and $r (t)$. An example where this is useful, is if one wants to not constraint theta, for example if one simply want to get to a higher orbit and does not care at what phase angle in that orbit. Always put to `false` if coordinates are not polar. 


In [9]:
import tensorflow as tf
import deepxde as dde
import numpy as np

class Dynamics(Function): 
    
    theta = False
    control = True
    mass = True
    entries = 6
    control_entries = 2
    loss_entries = 4
    coordinates = 'radial'
    loss_labels = ["r", "v$_r$", r"v$_{\theta}$", "m"]
    entry_labels = ["r", r"$\theta$", "v$_r$ ", r"v$_{\theta}$", "u$_r$", r"u$_{\theta}$"]
    
    def call(self, time, y):

            # Coordinate entries
            x1    = y[:, 0:1]
            theta = y[:,1:2] # Not used, uncoupled from the others
            x2    = y[:, 2:3]
            x3    = y[:, 3:4]

            # Control entries
            ur    = y[:, 4:5]
            ut    = y[:, 5:6]

            # Mass entry
            m     = y[:, 6:7]

            # Thrust magnitude
            T = tf.reshape(tf.norm(y[:, 4:6], axis = 1), (-1, 1))

            # LHS of equations of motion (Automatic differentation)
            dx1_dt    = dde.grad.jacobian(y, time, i=0)
            dtheta_dt = dde.grad.jacobian(y, time, i=1)
            dx2_dt    = dde.grad.jacobian(y, time, i=2)
            dx3_dt    = dde.grad.jacobian(y, time, i=3)
            dm_dt     = dde.grad.jacobian(y, time, i=6)

            # RHS of equations of motion
            RHS_x1     = x2
            RHS_theta  = x3/x1
            RHS_x2     = x3**2/x1 - (self.mu * self.time_scale**2 / self.length_scale**3) * x1**(-2) +\
                          (self.time_scale**2/self.length_scale) * ur / m
            RHS_x3     = - (x2*x3)/x1 + (self.time_scale**2/self.length_scale) * ut / m
            RHS_m      = -T * self.time_scale / (self.isp * 9.81)

            # Return the residuals
            return [
                dx1_dt    - RHS_x1,
                dtheta_dt - RHS_theta,
                dx2_dt    - RHS_x2,
                dx3_dt    - RHS_x3,
                dm_dt     - RHS_m,
                ]
        